In [5]:
install.packages("data.table")
install.packages("rlist")
install.packages("rpart")
install.packages("parallel")
install.packages("DiceKriging")
install.packages("mlrMBO")

Installing package into 'C:/Users/Tomás García/Documents/R/win-library/3.6'
(as 'lib' is unspecified)



  There is a binary version available but the source version is later:
           binary source needs_compilation
data.table 1.14.0 1.14.2              TRUE

  Binaries will be installed


Warning message:
"package 'data.table' is in use and will not be installed"Installing package into 'C:/Users/Tomás García/Documents/R/win-library/3.6'
(as 'lib' is unspecified)



  There is a binary version available but the source version is later:
       binary  source needs_compilation
rlist 0.4.6.1 0.4.6.2             FALSE



installing the source package 'rlist'

Warning message in install.packages("rlist"):
"installation of package 'rlist' had non-zero exit status"Installing package into 'C:/Users/Tomás García/Documents/R/win-library/3.6'
(as 'lib' is unspecified)



  There is a binary version available but the source version is later:
      binary source needs_compilation
rpart 4.1-15 4.1.16              TRUE

  Binaries will be installed


Warning message:
"package 'rpart' is in use and will not be installed"Installing package into 'C:/Users/Tomás García/Documents/R/win-library/3.6'
(as 'lib' is unspecified)
Warning message:
"package 'parallel' is not available (for R version 3.6.1)"Warning message:
"package 'parallel' is a base package, and should not be updated"Installing package into 'C:/Users/Tomás García/Documents/R/win-library/3.6'
(as 'lib' is unspecified)
Warning message:
"package 'DiceKriging' is in use and will not be installed"Installing package into 'C:/Users/Tomás García/Documents/R/win-library/3.6'
(as 'lib' is unspecified)
also installing the dependency 'parallelMap'




  There is a binary version available but the source version is later:
            binary source needs_compilation
parallelMap  1.5.0  1.5.1             FALSE

package 'mlrMBO' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Tomás García\AppData\Local\Temp\Rtmp0s2L4P\downloaded_packages


installing the source package 'parallelMap'

Warning message in install.packages("mlrMBO"):
"installation of package 'parallelMap' had non-zero exit status"

In [11]:
#Optimizacion Bayesiana de hiperparametros de  rpart

#limpio la memoria
rm( list=ls() )  #remove all objects
gc()             #garbage collection

require("data.table")
require("rlist")

require("rpart")
require("parallel")

#paquetes necesarios para la Bayesian Optimization
require("DiceKriging")
require("mlrMBO")


#Defino la  Optimizacion Bayesiana

kBO_iter  <- 100   #cantidad de iteraciones de la Optimizacion Bayesiana

# Aquí se define como va a ser la optimización bayesiana, le digo como va a aiterar por los parámetros
# forbidden es la condición que le pongo, de que un hijo no puede ser más que la mitad del padre
hs  <- makeParamSet(
          makeNumericParam("cp"       , lower= -1   , upper=    0.1),
          makeIntegerParam("minsplit" , lower=  1L  , upper= 8000L),  #la letra L al final significa ENTERO
          makeIntegerParam("minbucket", lower=  1L  , upper= 2000L),
          makeIntegerParam("maxdepth" , lower=  3L  , upper=   20L),
          forbidden = quote( minbucket > 0.5*minsplit ) )             # minbuket NO PUEDE ser mayor que la mitad de minsplit

# Mientras más parámetros tenga acá, más iteraciones tengo que mostrarle

ksemilla_azar  <- 102191   #cambiar por la primer semilla

#------------------------------------------------------------------------------
#graba a un archivo los componentes de lista
#para el primer registro, escribe antes los titulos

loguear  <- function( reg, arch=NA, folder="./work/", ext=".txt", verbose=TRUE )
{
  archivo  <- arch
  if( is.na(arch) )  archivo  <- paste0( folder, substitute( reg), ext )

  if( !file.exists( archivo ) )  #Escribo los titulos
  {
    linea  <- paste0( "fecha\t", 
                      paste( list.names(reg), collapse="\t" ), "\n" )

    cat( linea, file=archivo )
  }

  linea  <- paste0( format(Sys.time(), "%Y%m%d %H%M%S"),  "\t",     #la fecha y hora
                    gsub( ", ", "\t", toString( reg ) ),  "\n" )

  cat( linea, file=archivo, append=TRUE )  #grabo al archivo

  if( verbose )  cat( linea )   #imprimo por pantalla
}
#------------------------------------------------------------------------------
#particionar agrega una columna llamada fold a un dataset que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30), agrupa=clase_ternaria, seed=semilla)   crea una particion 70, 30 
# particionar( data=dataset, division=c(1,1,1,1,1), agrupa=clase_ternaria, seed=semilla)   divide el dataset en 5 particiones

particionar  <- function( data, division, agrupa="", campo="fold", start=1, seed=NA )
{
  if( !is.na( seed)  )   set.seed( seed )

  bloque  <- unlist( mapply(  function(x,y) { rep( y, x ) }, division, seq( from=start, length.out=length(division) )  ) )

  data[ , (campo) :=  sample( rep( bloque, ceiling(.N/length(bloque))) )[1:.N],
           by= agrupa ]
}
#------------------------------------------------------------------------------
#fold_test  tiene el numero de fold que voy a usar para testear, entreno en el resto de los folds
#param tiene los hiperparametros del arbol

ArbolSimple  <- function( fold_test, data, param )
{
  #genero el modelo
  modelo  <- rpart("clase_ternaria ~ .", 
                   data= data[ fold != fold_test, ],  #entreno en todo MENOS el fold_test que uso para testing
                   xval= 0,
                   control= param )

  #aplico el modelo a los datos de testing
  prediccion  <- predict( modelo, 
                          data[ fold==fold_test, ],  #aplico el modelo sobre los datos de testing
                          type= "prob")   #quiero que me devuelva probabilidades

  prob_baja2  <- prediccion[, "BAJA+2"]  #esta es la probabilidad de baja

  #calculo la ganancia
  ganancia_testing  <- data[ fold==fold_test ][ prob_baja2 > 1/60,  
                                                sum( ifelse( clase_ternaria=="BAJA+2", 59000, -1000 ) )] 

  return( ganancia_testing )  #esta es la ganancia sobre el fold de testing, NO esta normalizada
}
#------------------------------------------------------------------------------

ArbolesCrossValidation  <- function( data, param, qfolds, pagrupa, semilla )
{
  divi  <- rep( 1, qfolds )  # generalmente  c(1, 1, 1, 1, 1 )  cinco unos

  particionar( data, divi, seed=semilla, agrupa=pagrupa )  #particiono en dataset en folds

  ganancias  <- mcmapply( ArbolSimple, 
                          seq(qfolds), # 1 2 3 4 5
                          MoreArgs= list( data, param), 
                          SIMPLIFY= FALSE,
                          mc.cores= 1 )   #se puede subir a qfolds si posee Linux o Mac OS

  data[ , fold := NULL ]

  #devuelvo la primer ganancia y el promedio
  ganancia_promedio  <- mean( unlist( ganancias ) )   #promedio las ganancias
  ganancia_promedio_normalizada  <- ganancia_promedio * qfolds  #aqui normalizo la ganancia
  
  return( ganancia_promedio_normalizada )
}
#------------------------------------------------------------------------------
#esta funcion solo puede recibir los parametros que se estan optimizando
#el resto de los parametros, lamentablemente se pasan como variables globales

EstimarGanancia  <- function( x )
{
   GLOBAL_iteracion  <<-  GLOBAL_iteracion + 1

   xval_folds  <- 5
   ganancia  <- ArbolesCrossValidation( dataset,
                                        param= x, #los hiperparametros del arbol
                                        qfolds= xval_folds,  #la cantidad de folds
                                        pagrupa= "clase_ternaria",
                                        semilla= ksemilla_azar )

   #logueo 
   xx  <- x
   xx$xval_folds  <-  xval_folds
   xx$ganancia  <- ganancia
   xx$iteracion <- GLOBAL_iteracion
   loguear( xx,  arch= archivo_log )

   return( ganancia )
}
#------------------------------------------------------------------------------
#Aqui empieza el programa

setwd("C:\\Users\\Tomás García\\Documents\\MD")

#cargo el dataset
dataset  <- fread("./datasets/paquete_premium_202011.csv")   #donde entreno


#creo la carpeta donde va el experimento
# HT  representa  Hiperparameter Tuning
dir.create( "./labo/exp/",  showWarnings = FALSE ) 
dir.create( "./labo/exp/HT3210/", showWarnings = FALSE )
setwd("C:\\Users\\Tomás García\\Documents\\MD\\labo\\exp\\HT3210\\")   #Establezco el Working Directory DEL EXPERIMENTO


archivo_log  <- "HT321.txt"
archivo_BO   <- "HT321.RDATA"

#leo si ya existe el log, para retomar en caso que se se corte el programa
GLOBAL_iteracion  <- 0

if( file.exists(archivo_log) )
{
 tabla_log  <- fread( archivo_log )
 GLOBAL_iteracion  <- nrow( tabla_log )
}



#Aqui comienza la configuracion de la Bayesian Optimization

funcion_optimizar  <- EstimarGanancia

configureMlr( show.learner.output= FALSE)

# esta

#configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
#por favor, no desesperarse por lo complejo
obj.fun  <- makeSingleObjectiveFunction(
              fn=       funcion_optimizar,
              minimize= FALSE,   #estoy Maximizando la ganancia
              noisy=    TRUE,
              par.set=  hs,
              has.simple.signature = FALSE
             )

ctrl  <- makeMBOControl( save.on.disk.at.time= 600,  save.file.path= archivo_BO)
ctrl  <- setMBOControlTermination(ctrl, iters= kBO_iter )
ctrl  <- setMBOControlInfill(ctrl, crit= makeMBOInfillCritEI())

surr.km  <- makeLearner("regr.km", predict.type= "se", covtype= "matern3_2", control= list(trace= TRUE))

# esta última parte se fija si existe el archivo mbo, y retoma desde donde dejó la última corrida
#inicio la optimizacion bayesiana
if( !file.exists( archivo_BO ) ) {
  run  <- mbo(obj.fun, learner = surr.km, control = ctrl)
} else  run  <- mboContinue( archivo_BO )   #retomo en caso que ya exista

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,648273,34.7,1414170,75.6,1414170,75.6
Vcells,1265525,9.7,24837126,189.5,35698022,272.4


Loading required package: rlist
Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
"there is no package called 'rlist'"Loading required package: mlrMBO
Warning message:
"package 'mlrMBO' was built under R version 3.6.3"Loading required package: mlr
Warning message:
"package 'mlr' was built under R version 3.6.3"Error: package or namespace load failed for 'mlr' in loadNamespace(i, c(lib.loc, .libPaths()), versionCheck = vI[[i]]):
 there is no package called 'ggplot2'


ERROR: Error in configureMlr(show.learner.output = FALSE): no se pudo encontrar la función "configureMlr"


In [7]:
install.packages("mlr")

Installing package into 'C:/Users/Tomás García/Documents/R/win-library/3.6'
(as 'lib' is unspecified)
also installing the dependencies 'lifecycle', 'R6', 'RColorBrewer', 'rlang', 'scales', 'withr', 'ggplot2', 'parallelMap'




  There are binary versions available but the source versions are later:
             binary source needs_compilation
lifecycle     1.0.0  1.0.1             FALSE
R6            2.5.0  2.5.1             FALSE
RColorBrewer  1.1-2  1.1-3             FALSE
rlang        0.4.11  1.0.2              TRUE
scales        1.1.1  1.2.0             FALSE
withr         2.4.2  2.5.0             FALSE
ggplot2       3.3.3  3.3.6             FALSE
parallelMap   1.5.0  1.5.1             FALSE

  Binaries will be installed
package 'rlang' successfully unpacked and MD5 sums checked
package 'mlr' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Tomás García\AppData\Local\Temp\Rtmp0s2L4P\downloaded_packages


installing the source packages 'lifecycle', 'R6', 'RColorBrewer', 'scales', 'withr', 'ggplot2', 'parallelMap'

Warning message in install.packages("mlr"):
"installation of package 'lifecycle' had non-zero exit status"Warning message in install.packages("mlr"):
"installation of package 'R6' had non-zero exit status"Warning message in install.packages("mlr"):
"installation of package 'RColorBrewer' had non-zero exit status"Warning message in install.packages("mlr"):
"installation of package 'withr' had non-zero exit status"Warning message in install.packages("mlr"):
"installation of package 'parallelMap' had non-zero exit status"Warning message in install.packages("mlr"):
"installation of package 'scales' had non-zero exit status"Warning message in install.packages("mlr"):
"installation of package 'ggplot2' had non-zero exit status"

In [ ]:
install.packages("mlrMBO")